<a href="https://colab.research.google.com/github/vrose99/ds4bme/blob/master/Quiz4_vrose9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Victoria Rose
### vrose9@jh.edu
### Quiz 4

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import sklearn as skl
import statsmodels.formula.api as smf
import statsmodels as sm

# Question 1

In [139]:
# Download in the data if it's not already there
! if [ ! -e kirby21.csv ]; \
then wget https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/kirby21.csv; \
fi;

# Read in the data and display a few rows
dat = pd.read_csv("kirby21.csv")
#dat['csf'] = dat['icv'] - dat['tbv'] 
dat.head()

,Unnamed: 0,rawid,roi,volume,min,max,mean,std,type,level,id,icv,tbv
0,1,kirby127a_3_1_ax.img,Telencephalon_L,531111,0.0,374.0,128.3013,51.8593,1,1,127,1378295,1268519
1,2,kirby127a_3_1_ax.img,Telencephalon_R,543404,0.0,300.0,135.0683,53.6471,1,1,127,1378295,1268519
2,3,kirby127a_3_1_ax.img,Diencephalon_L,9683,15.0,295.0,193.5488,32.2733,1,1,127,1378295,1268519
3,4,kirby127a_3_1_ax.img,Diencephalon_R,9678,10.0,335.0,193.7051,32.7869,1,1,127,1378295,1268519
4,5,kirby127a_3_1_ax.img,Mesencephalon,10268,55.0,307.0,230.8583,29.2249,1,1,127,1378295,1268519


In [140]:
# Make a data Frame
df = pd.read_csv("kirby21.csv")
df_L = df.loc[df['roi']=='Diencephalon_L']
df_R = df.loc[df['roi']=='Diencephalon_R']
df_y = df.loc[df['roi']=='CSF']
#df_L.head()
#df_R.head()
x1 = df_L['volume'].tolist()
x2 = df_R['volume'].tolist()
y_temp = df_y['volume'].tolist()
y = []
for i in range(len(y_temp)):
  if i % 2 == 0:
    y.append(y_temp[i])
#will be the same whether taken from df_L or df_R
#pred = pd.DataFrame([x1, x2, y],columns=['x1','x2','y'])
df = pd.DataFrame([x1,x2,y])
df = df.transpose()
df.columns = ['x1','x2','y']
df.head()

,x1,x2,y
0,9683,9678,109776
1,9156,9071,107334
2,8937,9004,94254
3,8828,8933,67156
4,9749,9722,110114


In [141]:
results = smf.ols('y ~ x1+ x2', data = df).fit()
print(results.summary2())

                    Results: Ordinary least squares
Model:                OLS               Adj. R-squared:      0.128     
Dependent Variable:   y                 AIC:                 454.0017  
Date:                 2019-09-27 02:23  BIC:                 456.9889  
No. Observations:     20                Log-Likelihood:      -224.00   
Df Model:             2                 F-statistic:         2.394     
Df Residuals:         17                Prob (F-statistic):  0.121     
R-squared:            0.220             Scale:               3.6842e+08
-----------------------------------------------------------------------
            Coef.     Std.Err.     t    P>|t|     [0.025       0.975]  
-----------------------------------------------------------------------
Intercept -2449.7146 59324.5017 -0.0413 0.9675 -127613.4723 122714.0432
x1           23.9403    21.0337  1.1382 0.2708     -20.4370     68.3175
x2          -13.7653    23.3248 -0.5902 0.5628     -62.9763     35.4458
------------

### Later problem interpret

# Question 2

In [0]:
# Download in the data if it's not already there
! if [ ! -e binary.csv ]; \
then wget https://stats.idre.ucla.edu/stat/data/binary.csv; \
fi;

In [143]:
dat = pd.read_csv("binary.csv")
dat.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [167]:
trainFraction = .75
x = dat[['gre','gpa','rank']]
y = dat[['admit']]
x = sm.tools.add_constant(x)
sample = np.random.uniform(size = 400) < trainFraction
trainingDat = dat[sample]
testingDat = dat[~sample]
xtraining = x[sample]
xtesting = x[~sample]
ytraining = y[sample]
ytesting = y[~sample]
model = sm.discrete.discrete_model.Logit(ytraining, xtraining).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.569924
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  305
Model:                          Logit   Df Residuals:                      301
Method:                           MLE   Df Model:                            3
Date:                Fri, 27 Sep 2019   Pseudo R-squ.:                 0.08867
Time:                        02:39:17   Log-Likelihood:                -173.83
converged:                       True   LL-Null:                       -190.74
Covariance Type:            nonrobust   LLR p-value:                 2.155e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1588      1.321     -3.149      0.002      -6.747      -1.570
gre            0.0027      0.

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## INTERP COEFF

In [0]:
thresh = np.arange(0,1,0.01).tolist()
yhat = model.predict(xtesting).tolist()
ytesting = ytesting.values
yhat_temp = np.zeros(len(yhat))
results = np.zeros((len(thresh),4))
for j in range(len(thresh)):
  true_pos = 0
  true_neg = 0
  false_pos = 0
  false_neg = 0
  for k in range(len(yhat)):
    if yhat[k] >= thresh[j]:
      yhat_temp[k] = 1
    else:
      yhat_temp[k] = 0
  for i in range(len(yhat)):
    if (yhat_temp[i] == 1) and (ytesting[i] == 1): #true pos
      true_pos += 1
    elif (yhat_temp[i] == 1) and (ytesting[i] == 0): 
      false_pos += 1
    elif (yhat_temp[i] == 0) and (ytesting[i] == 0):
      true_neg += 1
    else:
      false_neg += 1
  results[j,0] = true_pos
  results[j,1] = false_pos
  results[j,2] = true_neg
  results[j,3] = false_neg
    

array([ 0.,  0.,  0.,  1.,  1.,  1.,  1.,  4.,  6.,  7.,  9., 12., 13.,
       17., 17., 19., 21., 21., 21., 22., 23., 23., 24., 24., 24., 24.,
       27., 30., 32., 33., 35., 36., 38., 39., 41., 43., 45., 46., 46.,
       47., 47., 48., 48., 50., 50., 50., 50., 51., 53., 53., 53., 53.,
       53., 53., 53., 53., 54., 54., 54., 54., 55., 55., 55., 55., 55.,
       56., 56., 56., 56., 57., 57., 57., 57., 58., 58., 58., 58., 59.,
       59., 59., 59., 59., 59., 59., 59., 59., 59., 59., 59., 59., 59.,
       59., 59., 59., 59., 59., 59., 59., 59., 59.])

In [0]:
yhat

0      0.244743
2      0.700974
3      0.138303
4      0.092614
9      0.532472
10     0.338035
11     0.376114
13     0.294425
14     0.675616
24     0.382179
25     0.609577
28     0.351591
29     0.418774
30     0.224008
41     0.325413
51     0.105490
52     0.183015
53     0.338378
57     0.127005
60     0.299632
64     0.395513
66     0.231969
69     0.629298
70     0.412595
80     0.108597
84     0.269549
86     0.330626
93     0.232334
95     0.349223
100    0.151358
         ...   
256    0.304946
259    0.481918
261    0.250162
269    0.125515
274    0.261341
276    0.301323
279    0.299524
280    0.526645
287    0.232153
306    0.389932
314    0.164522
315    0.163306
316    0.198565
317    0.242692
320    0.168815
322    0.098843
323    0.158259
327    0.363223
347    0.461264
350    0.579395
353    0.413855
358    0.306106
360    0.627349
366    0.136453
376    0.422831
377    0.585147
387    0.335995
388    0.302092
390    0.311824
395    0.532729
Length: 97, dtype: float